In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [89]:
taxi_zones_csv = pd.read_csv("../data/input/taxi_zones/taxi_zones.csv", index_col='LocationID')
taxi_zones_geojson = pd.read_json("../data/input/taxi_zones/taxi_zones.geojson")

In [90]:
taxi_zones_csv

,Borough,Zone,service_zone
LocationID,,,
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
...,...,...,...
261,Manhattan,World Trade Center,Yellow Zone
262,Manhattan,Yorkville East,Yellow Zone
263,Manhattan,Yorkville West,Yellow Zone


In [103]:
# taxi_zones_csv.groupby('Brooklyn', axis=0)
taxi_zones_fil = taxi_zones_csv.loc[taxi_zones_csv['Borough'] == 'Brooklyn'].drop('Borough', axis=1).drop('service_zone', axis=1)
taxi_zones_fil

,Zone
LocationID,
11,Bath Beach
14,Bay Ridge
17,Bedford
21,Bensonhurst East
22,Bensonhurst West
...,...
227,Sunset Park East
228,Sunset Park West
255,Williamsburg (North Side)


In [127]:
taxi_zones_geojson['features'][0]['properties']['borough'] == 'Brooklyn'


False

In [128]:
taxi_zones_geojson

,type,name,features
0,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
1,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
2,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
3,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
4,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
...,...,...,...
258,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
259,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
260,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."
261,FeatureCollection,taxi_zones,"{'type': 'Feature', 'properties': {'OBJECTID':..."


In [125]:
# access coordinates
taxi_zones_geojson['features'][0]['geometry']['coordinates'][0][0]

[933100.9183527103, 192536.08569720192]

In [29]:
# read in trip data
trip_data1 = pd.read_parquet("../data/input/trip_data/01.parquet")
trip_data2 = pd.read_parquet("../data/input/trip_data/02.parquet")
trip_data3 = pd.read_parquet("../data/input/trip_data/03.parquet")
trip_data4 = pd.read_parquet("../data/input/trip_data/04.parquet")
trip_data5 = pd.read_parquet("../data/input/trip_data/05.parquet")
trip_data6 = pd.read_parquet("../data/input/trip_data/06.parquet")
trip_data7 = pd.read_parquet("../data/input/trip_data/07.parquet")
trip_data8 = pd.read_parquet("../data/input/trip_data/08.parquet")
trip_data9 = pd.read_parquet("../data/input/trip_data/09.parquet")
trip_data10 = pd.read_parquet("../data/input/trip_data/10.parquet")
trip_data11 = pd.read_parquet("../data/input/trip_data/11.parquet")
trip_data12 = pd.read_parquet("../data/input/trip_data/12.parquet")

# had to install pyarrow via pip install and fastparquet

In [122]:
taxi_zones_geojson.columns

Index(['type', 'name', 'features'], dtype='object')

In [6]:
trip_data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6339567 entries, 0 to 6339566
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[ns]
 1   tpep_dropoff_datetime  datetime64[ns]
 2   passenger_count        float64       
 3   trip_distance          float64       
 4   RatecodeID             float64       
 5   PULocationID           int64         
 6   DOLocationID           int64         
 7   payment_type           float64       
 8   fare_amount            float64       
 9   extra                  float64       
 10  mta_tax                float64       
 11  tip_amount             float64       
 12  tolls_amount           float64       
 13  improvement_surcharge  float64       
 14  total_amount           float64       
 15  congestion_surcharge   float64       
dtypes: datetime64[ns](2), float64(12), int64(2)
memory usage: 822.2 MB


## 1 Exploration and Description
### a) Explain the data types in different columns of your dataset. Are all numeric columns actually numbers, or are there categories?
https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf
Technically the data types are numerical except for the pickup_datetime and the dropoff_datetime. 
But on a detailed look there are some categories:

    PULocationID: TLC Taxi Zone in which the taximeter was engaged
    
    DOLocationID: TLC Taxi Zone in which the taximeter was disengaged
    
    RateCodeID:   The final rate code in effect at the end of the trip.
                    1 = Standard rate
                    2 = JFK
                    3 = Newark
                    4 = Nassau or Westchester
                    5 = Negotiated fare
                    6 = Group ride
                    
    payment_type: Payment_type A numeric code signifying how the passenger paid for the trip.
                    1 = Credit card
                    2 = Cash
                    3 = No charge
                    4 = Dispute
                    5 = Unknown
                    6 = Voided trip
                    
    fare_amout:   The time-and-distance fare calculated by the meter.
    
    ...

In [35]:
trip_data4["RatecodeID"].unique()

array([ 1.,  5.,  4.,  2.,  3., 99.,  6.])

### b) In the dataset, you find the two columns “PULocationID” and “DOLocationID”. What exactly do they describe? Using the provided taxi_zones data and respective geo data packages, find the centers of the respective areas. (You might need some research to find the right functions, if you are stuck here, contact me).

In [36]:
import folium

In [53]:
tiles='https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png'
attr="Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL."
m = folium.Map(location = [40.650002, -73.949997], zoom_start=12, attr=attr, tiles=tiles)
m

In [129]:
folium.Choropleth(
    geo_data=f"data/states.geojson",  # geojson file with spatial/geographical information
    data = taxi_zones_csv,
    columns = ["States", "Population"],
    key_on="properties.NAME_1", # connect dataframe and geojson information
    fill_color='PuBuGn', # Find more here: https://github.com/dsc/colorbrewer-python
    fill_opacity = 0.5,
    line_opacity = 0.2, 
    legend_name="Population [Mio.]").add_to(m)

m

NameError: name 'states_df' is not defined